In [1]:
import numpy as np
from scipy.linalg import eigh
from scipy.integrate import solve_ivp
from pydub import AudioSegment
from pydub.playback import play

# Constants
m = 1.0           # Mass of the particle
omega = 2 * np.pi * 440  # Angular frequency of oscillator (440 Hz, A4 note)
hbar = 1.0        # Planck's reduced constant
N = 50            # Number of basis states
dt = 0.01         # Time step
duration = 5.0    # Duration of the sound (seconds)
fs = 44100        # Sampling rate (Hz)

# Define the QHO Hamiltonian
def quantum_harmonic_oscillator(N, m, omega, hbar):
    """Generate the Hamiltonian for a Quantum Harmonic Oscillator."""
    H = np.zeros((N, N))
    for n in range(N):
        if n > 0:
            H[n, n-1] = H[n-1, n] = np.sqrt(n / 2) * hbar * omega
        H[n, n] = (n + 0.5) * hbar * omega
    return H

# Initial State
def initial_state(N):
    """Define an initial state as a superposition of low energy eigenstates."""
    state = np.zeros(N)
    state[0] = 1.0  # Ground state
    return state / np.linalg.norm(state)

# Time Evolution
def evolve_state(H, psi0, t):
    """Evolve the state using the Hamiltonian."""
    eigenvalues, eigenvectors = eigh(H)
    exp_terms = np.exp(-1j * eigenvalues * t / hbar)
    evolved_state = eigenvectors @ (exp_terms * (eigenvectors.T @ psi0))
    return evolved_state

# Generate Sound Signal
def generate_signal(H, psi0, duration, dt, fs):
    """Generate a sound signal from the wavefunction evolution."""
    t_samples = np.arange(0, duration, 1/fs)
    signal = np.zeros_like(t_samples)
    for i, t in enumerate(t_samples):
        psi_t = evolve_state(H, psi0, t)
        signal[i] = np.real(np.vdot(psi_t, psi_t))  # Measure probability density
    return signal

# Main script
H = quantum_harmonic_oscillator(N, m, omega, hbar)
psi0 = initial_state(N)
signal = generate_signal(H, psi0, duration, dt, fs)

# Normalize signal for audio
signal = (signal - np.min(signal)) / (np.max(signal) - np.min(signal))  # Normalize to [0, 1]
signal = (signal * 2 - 1) * 32767  # Convert to 16-bit PCM

# Create a WAV file
from scipy.io.wavfile import write
write("quantum_oscillator_sound.wav", fs, signal.astype(np.int16))

# Play the sound
audio = AudioSegment(
    signal.astype(np.int16).tobytes(),
    frame_rate=fs,
    sample_width=2,
    channels=1
)
play(audio)


ModuleNotFoundError: No module named 'pydub'